ELEC-E5550 - Statistical Natural Language Processing
# SET 6: Subword segmentation

## Released: 28.02.2023 at 14:30
## Deadline: 13.03.2023 at midnight

# Overview

We've already talked about the problem of segmenting text into appropriate units (tokenization). Back then, it was words that were considered as those units, but there are other units that you should probably explore as well: characters and **subwords**. In this assignment, we're going to focus on **tokenization into subwords**.

The motivation to segmenting words further into smaller elements comes from morphology, where such elements are called **morphemes**. A **morpheme** is defined as the smallest meaning-bearing unit of a language. For example, the word *unpredictable* contains three morphemes: *un*, *predict* and *able*. As you can see, many morphemes are not unique to one word, they are elements that are regularly seen in other words too. For example, *un* in *unhappy*, *predict* in *predictive*, and *able* in *comfortable*. Thus, just as sentences are constructed from words, words are constructed from morphemes. 

Segmenting into morphemes (especially in languages with rich morphology) helps to avoid the problem of out-of-vocabulary (OOV) words in text corpora. For example, if our training corpus contains *cool*, *cool-est* and *dumb-er* when the new word *cooler* comes in, it can be segmented into *cool-er* and understood based on its constituent subwords.

# Table of contents

* [Task 1: BPE](#task_1)
    * [Step 1.1](#subtask_1_1)
        * [Step 1.1.1: Collecting word counts](#subtask_1_1_1)
        * [Step 1.1.2: Convering words to characters](#subtask_1_1_2)
    * [Step 1.2: Collecting symbol pairs frequencies](#subtask_1_2)
    * [Step 1.3: Merging the most frequent pair](#subtask_1_3)
    * [Step 1.4: Combining steps 1-3 together](#subtask_1_4)
    * [Step 1.5-7: Segmenting a corpus](#subtask_1_5)
* [Task 2: ANALYZE SEGMENTATIONS](#task_2)
    * [Step 2.1: Count word OOV](#subtask_2_1)
    * [Step 2.2: Count subword OOV](#subtask_2_2)
    * [Step 2.3: Does the segmentation make sense?](#subtask_2_3)
    * [Step 2.4: Your thoughts](#subtask_2_4)
* [Checklist](#checklist)

## TASK 1 <a class="anchor" id="task_1"></a>
## BPE
One approach to tokenization into subwords is based on the **byte pair encoding** (**BPE**) algorithm for data compression. Starting from single characters as the subwords, it iteratively merges frequent pairs of subwords forming new subwords. Assuming morphemes are frequently repeated substrings, this method should often merge symbols into morphemes instead of into other, meaningless character sequences. The algorithm is applied only inside words (there are no merges across word boundaries). 

**BPE** algorithm begins with its vocabulary being a set of characters seen in the training corpus. Each word in the corpus is represented as a sequence of characters plus a special end-of-word symbol '_'. At each iteration step $k_i$, the algorithm counts the number of symbol pairs, finds the most frequent pair ['A','B'] and replaces it with the new merged symbol ‘AB’. The algorithm stops when it's done $k$ merges ($k$ is a parameter of the algorithm). The algorithm begins with the set of symbols equal to the set of characters. The resulting symbol set should have the original set of characters plus $k$ new symbols. 

To obtain a subword vocabulary with **BPE**, you should take the following steps:
* STEP 1: tokenize a training corpus into words and collect frequency statistics of word tokens in the training corpus. Additionally, represent each word as a list of characters plus a special end-of-word symbol '_'.
* STEP 2: count the frequencies of symbol pairs.
* STEP 3: replace every occurrence of the most frequent pair ['A', 'B'] with the new merged symbol 'AB'.
* STEP 4: repeat STEPs 2-3 $k-1$ times more.

To segment a corpus using the vocabulary, you should:
* STEP 5: tokenize a test corpus into words (with the same tokenization algorithm as in training).
* STEP 6: represent each word as a list of characters plus a special end-of-word symbol '_'.
* STEP 7: for every word apply each merge operation in the order they were learned, and return the tokenized text

## 1.1 <a class="anchor" id="subtask_1_1"></a>
### STEP 1
### 1.1.1  <a class="anchor" id="subtask_1_1_1"></a>
### Collecting word counts (1 Point)

Write a function that reads a text as one string from a file, tokenizes it into words by whitespaces and collects the word frequencies. It should return a dictionary of words and their raw counts in a corpus.

In [22]:
import collections
def collect_word_counts(file_name):
    """
    Takes in a path to a text file, reads the file, splits it into words by whitespaces, 
    and then counts the words' frequencies.
    
    Parameters
    ---------
    file_name : string
            a path to a training corpus as a string
    
    Returns
    -------
    word_counts : dictionary
            a dictionary of word counts
    """
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    fp=open(file_name,'r')
    text=fp.read()
    fp.close()
    tokens=text.split()
    dic=collections.Counter(tokens)
    

   
    word_counts=dict(dic)
    
    return word_counts

In [23]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_corpus_path = "/coursedata/06-subwords/dummy_corpus.txt"

# check that the output of the function is a list
assert_equal(type(collect_word_counts(dummy_corpus_path)), dict)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD
assert_equal(collect_word_counts(dummy_corpus_path)['low'], 5)
assert_equal(collect_word_counts(dummy_corpus_path)['lowest'], 2)
assert_equal(collect_word_counts(dummy_corpus_path)['new'], 2)
assert_equal(collect_word_counts(dummy_corpus_path)['newer'], 6)
assert_equal(collect_word_counts(dummy_corpus_path)['WIDer'], 3)


# A SANITY CHECK FOR THE NOTEBOOK DATA
gum_train_path = "/coursedata/06-subwords/gum_train.txt"
# check that the vocabulary length is right
assert_equal(len(collect_word_counts(gum_train_path)), 8677)
# check that the word count is right
assert_equal(collect_word_counts(gum_train_path)['we'], 112)



### 1.1.2  <a class="anchor" id="subtask_1_1_1"></a>
### Converting words to characters (1 Point)
Now, represent each word in your frequency dictionary as a tuple of characters plus a special end-of-word marker '_'.

In [24]:
def convert_to_chars(vocab):
    """
    Takes in a frequeny dictionary of words in the training corpus
    and converts the key words to a tuple of characters plus a special end-of-word symbol '_'.
    
    Parameters
    ----------
    vocab : dictionary
        a frequency dictionary of words
    
    Returns
    -------
    separated_vocab : dictionary 
        a frequency dictionary of words represented as a tuple of characters 
        plus a special end-of-word symbol '_'
        {('l','o','w','_') : 3}
    """
   
    separated_vocab={}
    for key in vocab:
        newkey=[]
        for ch in key:
            newkey.append(ch)
        newkey.append('_')
        nk=tuple(newkey)
        separated_vocab[nk]=vocab[key]
    # YOUR CODE HERE
    #raise NotImplementedError()
  
    return separated_vocab

In [25]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_freq_vocab = {'low': 5, 'lowest': 2, 'newer': 6, 'wider': 3, 'New': 2}

# check that the output of the function is a dict
assert_equal(type(convert_to_chars(dummy_freq_vocab)), dict)
# check that the keys are tuples
assert_equal(type(list(convert_to_chars(dummy_freq_vocab).keys())[0]), tuple)
#check that there are no new elements
assert_equal(len(convert_to_chars(dummy_freq_vocab)), 5)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD

assert_equal(convert_to_chars(dummy_freq_vocab)[('l', 'o', 'w', '_')], 5)
assert_equal(convert_to_chars(dummy_freq_vocab)[('l', 'o', 'w', 'e', 's', 't', '_')], 2)
assert_equal(convert_to_chars(dummy_freq_vocab)[('n', 'e', 'w', 'e', 'r', '_')], 6)
assert_equal(convert_to_chars(dummy_freq_vocab)[('w', 'i', 'd', 'e', 'r', '_')], 3)
assert_equal(convert_to_chars(dummy_freq_vocab)[('N', 'e', 'w', '_')], 2)



## 1.2 <a class="anchor" id="subtask_1_2"></a>
### STEP 2
### Collecting symbol pairs frequencies (1 Point)
Write a function that takes in a frequency dictionary where keys are words represented as tuples of symbols, and outputs the most frequent pair of symbols in the corpus. In the case, when there are several pairs with the same frequency, return the pair that is earlier alphabetically. 

For example, if we only have one *l o o k _* and one *l o o p _*  in our frequency dictionary, the fuction should output *l o* as the most frequent pair (it is earlier than *o o* alphabetically).

In [26]:
def convertTuple(tup):
        # initialize an empty string
    str = ''
    for item in tup:
        str = str + item
    return str
 


def get_the_pair_to_merge(vocab_as_symbols):
    """
    Takes in a frequency dictionary, where keys are words represented as tuples of symbols and values are their counts, 
    and outputs the most frequent pair of symbols in the corpus.
    
    Arguments
    ---------
    vocab_as_symbols : dictionary
        a frequency dictionary, where keys are words represented as tuples of symbols and values are their counts
    
    Returns
    -------
    merge_pair : tuple of strings 
        the most frequent pair of symbols, a pair to merge
    """

    merge_pair=()
    list_couples=[]
   
    for val in vocab_as_symbols:
        count=vocab_as_symbols[val]
        for i in range(len(val)-1):
            c1=val[i]
            c2=val[i+1]
            tup=(c1,c2)
            for j in range(count):
                list_couples.append(tup)
    
    dic=collections.Counter(list_couples)
 
    
    
    maxv=-1
    listaz=[]
    for k in dic:
        mp=dic[k]
        if (mp>maxv):
            maxv=mp
            merge_pair=k
        elif(mp==maxv):
            str1=convertTuple(merge_pair)
            str2=convertTuple(k)
            if(k[0]<merge_pair[0]):
                #print(str2)
                #print(str1)
                merge_pair=k
      
    # YOUR CODE HERE
    #raise NotImplementedError()
    return merge_pair

In [27]:
# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_freq_vocab_as_symbols = {('l','o','o','k','_') : 1, 
                               ('l','o','o','p','_') : 1}

# check that the output of the function is a tuple
assert_equal(type(get_the_pair_to_merge(dummy_freq_vocab_as_symbols)), tuple)
# check that the output of the function is a tuple of strings
assert_equal(type(get_the_pair_to_merge(dummy_freq_vocab_as_symbols)[0]), str)


# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD

assert_equal(get_the_pair_to_merge(dummy_freq_vocab_as_symbols), ('l', 'o'))


dummy_freq_vocab_as_symbols2 = {('l', 'o', 'w', '_'): 5,
                                ('l', 'o', 'w', 'e', 's', 't', '_'): 2,
                                ('n', 'e', 'w', 'e', 'r', '_'): 6, 
                                ('w', 'i', 'd', 'e', 'r', '_'): 3, 
                                ('n', 'e', 'w', '_'): 2}


assert_equal(get_the_pair_to_merge(dummy_freq_vocab_as_symbols2), ('e', 'r'))

# check that your functions can merge not only characters

dummy_freq_vocab_as_symbols3 = {('lo', 'o', 'k', '_'): 1 , 
                                ('lo', 'o', 'p','_'): 1}

assert_equal(get_the_pair_to_merge(dummy_freq_vocab_as_symbols3), ('lo', 'o'))


## 1.3 <a class="anchor" id="subtask_1_3"></a>
### STEP 3
### Merging the most frequent pair (3 Points)

Write a function that takes in a pair of symbols to merge and a frequency dictionary where words are represented as tuples of symbols. It should return a frequency dictionary, where words are still represented as tuples of symbols but the most frequnt pair is now merged in every word.

For example, if we want to merge *l* and *o* in our *l o o k _* and *l o o p _*  frequency dictionary, the fuction should output *lo o k _* and *lo o p _*

In [28]:
def merge(vocab_as_symbols, merge_pair):
    """Merges the most frequent pair of symbols in all words
    
    Parameters
    ---------
    vocab_as_symbols : dictionary
        a frequency dictionary, where keys are words repreqsented as tuples of symbols and values are their counts
    merge_pair : tuple
        a pair of symbols to merge
        
    Returns
    -------
    new_vocab : dictionary
        a frequency dictionary, where keys are words represented as tuples of symbols, 
        with the given pair represented as a new symbol (concatenated pair)
    """
    new_vocab={}
    
    for k  in vocab_as_symbols:
        val=vocab_as_symbols[k]
        newk=[]
        i=0
        while(i<(len(k)-1)):
            ch1=k[i]
            ch2=k[i+1]
            tup=(ch1,ch2)
            if(tup==merge_pair):
                str1=convertTuple(merge_pair)
                newk.append(str1)
                i=i+len(tup)
                if(i==(len(k)-1)):
                    newk.append(k[len(k)-1])         
            else:
                newk.append(ch1)
                i=i+1 
                if(i==(len(k)-1)):
                    newk.append(ch2)
                    
            
   
        
        newkt=tuple(newk)
        new_vocab[newkt]=val           
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return new_vocab

In [29]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_vocab_as_symbols = {('l', 'o', 'w', '_'): 5,
                          ('l', 'o', 'w', 'e', 's', 't', '_'): 2,
                          ('n', 'e', 'w', 'e', 'r', '_'): 6, 
                          ('w', 'i', 'd', 'e', 'r', '_'): 3, 
                          ('n', 'e', 'w', '_'): 2}

dummy_pair = ('e', 'r')

# check that the output of the function is a dictionary
assert_equal(type(merge(dummy_vocab_as_symbols, dummy_pair)), dict)
# check that the keys are tuples
assert_equal(type(list(merge(dummy_vocab_as_symbols, dummy_pair).keys())[0]), tuple)
#check that there are no new elements
assert_equal(len(merge(dummy_vocab_as_symbols, dummy_pair)), 5)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD
# check that the pair was merged everywhere
assert_equal(list(merge(dummy_vocab_as_symbols, dummy_pair).keys()), [('l', 'o', 'w', '_'), 
                                                                    ('l', 'o', 'w', 'e', 's', 't', '_'), 
                                                                    ('n', 'e', 'w', 'er', '_'), 
                                                                    ('w', 'i', 'd', 'er', '_'), 
                                                                    ('n', 'e', 'w', '_')])
# check that you can handle several merge pairs in a word
dummy_vocab_as_symbols2 = {('l','o','o','o','l'): 1, 
                           ('l','o','o','o','o','l'):1}

dummy_pair2 = ('o', 'o')


assert_equal(list(merge(dummy_vocab_as_symbols2, dummy_pair2).keys()), [('l', 'oo', 'o', 'l'), 
                                                                        ('l','oo','oo','l')])

# check that you can handle merge pairs of multicharacter symols
dummy_vocab_as_symbols3 = {('l','oo','oo','l'): 1, 
                           ('l','oo','oo','oo','o','l'):1}

dummy_pair3 = ('oo', 'oo')


assert_equal(list(merge(dummy_vocab_as_symbols3, dummy_pair3).keys()), [('l', 'oooo', 'l'), 
                                                                        ('l','oooo','oo','o','l')])


## 1.4 <a class="anchor" id="subtask_1_4"></a>
### STEP 4
### Combining steps 1-3 together (1 Point)
Now let's combine steps 1-3 into a function that learns $k$ BPE merges.

In [30]:
def learn_BPE_merges(file_name, k):
    """ Learns k BPE subwords
    
    STEP 1: Collect a word count dictionary from a file,
            represent words as a tuple of their characters plus a special end-of-word symbol '_'
    Now k times
        STEP 2: Choose the most frequent pair of symbols
                add this pair as a new subword unit into a subword vocabulary
        STEP 3: Merge the symbols in all words

        
    
    Parameters
    ---------
    file_name : string
        a path to a training corpus as a string
    k : integer
        a number of merges to be learned
    
    Returns
    -------
    merges : list of strings
        a list of k subwords (symbol merges) in the order they were learned
        one merge is a tuple of two symbols in the most frequent pair at step k
    """
    ret1=collect_word_counts(file_name)
    ret2=convert_to_chars(ret1)
    merges = []
    for i in range(k):     
        merge_pair=get_the_pair_to_merge(ret2)    
        merges.append(merge_pair)
        ret2=merge(ret2, merge_pair)
     
        
    

    # YOUR CODE HERE
    #raise NotImplementedError()
    
    return merges

In [31]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_corpus_path = "/coursedata/06-subwords/dummy_corpus.txt"


# check that the output of the function is a list
assert_equal(type(learn_BPE_merges(dummy_corpus_path, 10)), list)
# check that the output of the function is a list of tuples
assert_equal(type(learn_BPE_merges(dummy_corpus_path, 10)[0]), tuple)
# check that the output of the function is a list of tuples of strings
assert_equal(type(learn_BPE_merges(dummy_corpus_path, 10)[0][0]), str)
#check that there are exactly k merges
assert_equal(len(learn_BPE_merges(dummy_corpus_path, 10)), 10)

# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD
# check that 8 dummy merges are correct
assert_equal(learn_BPE_merges(dummy_corpus_path, 8), [('e', 'r'),
                                                     ('er', '_'),
                                                     ('e', 'w'),
                                                     ('n', 'ew'),
                                                     ('l', 'o'),
                                                     ('lo', 'w'),
                                                     ('new', 'er_'),
                                                     ('low', '_')])

# A SANITY CHECK FOR THE NOTEBOOK DATA
gum_train_path = "/coursedata/06-subwords/gum_train.txt"
assert_equal(learn_BPE_merges(gum_train_path, 5), [('e', '_'),
                                                   ('s', '_'),
                                                   ('t', 'h'),
                                                   ('t', '_'),
                                                   ('d', '_')])

             

## 1.5-7 <a class="anchor" id="subtask_1_5"></a>
### STEPS 5-7
### Segmenting a corpus (5 Points)
Well, now we can apply what we've learned to tokenize any text into its subwords. Write a function that reads the test corpus and applies the merges we've learned on a training corpus. Note that you will need to adapt your previous functions a little for this. For example, we don't need to count the word frequencies since they don't play any role here anymore.

Just a reminder of the steps needed to apply a subword tokenzation to a new (or an old) text:

* STEP 5: tokenize a test corpus into words (with the same tokenization algorithm as in training).
* STEP 6: represent each word as a list of characters plus a special end-of-word symbol '_'.
* STEP 7: for every word, apply each merge operation in the order they were learned, remove the end-of-word symbol '_' and return the tokenized text.


For the purposes of the exercise, the tokenized text should be a list of strings where strings are words with their subwords separated by whitespaces: ['I', 'lo ok', 'g oo d']. This way it will be easier for you to check how each word is tokenized. In the real application, a tokenized text will be represented just as a list of subwords.


Note: don't forget to get rid of the special end-of-word symbol after tokenization!

In [32]:
def segment_text(file_name, merges):
    """
    Takes in a path to a text file and lerned BPE merges,
    reads the file and tokenizes it into subwords in accorance with the merges.
    
    Arguments
    ---------
    file_name : string
        a path to a text as a string
    merges : list of tuples
        a list of k merges in the order they were learned
    
    Returns
    -------
    segmented_text - list of strings
        a text segmented with BPE
        the text is a list of words
        where each word is a string with its segments separated by whitespaces:
        ['I', 'lo ok', 'g oo d']
    """
    listmerges=[]
    fp=open(file_name,'r')
    text=fp.read()
    fp.close()
    tokens=text.split()
    lista=[]
    for token in tokens:
        for ch in token:
            lista.append(ch)
        lista.append('_')
    
    segmented_text=[]
    sentence=[]
    dic={}
    for elem in lista:
        if(elem=='_'):
            sentence.append(elem)
            dic[tuple(sentence)]=1
            for m in merges:
                dic=merge(dic,m)
            stri=''
            for k in dic:
                for t in k:
                    if(stri==''):
                        stri=t
                    elif(t=='_'):
                        stri=stri
                    else:
                        stri=stri+' '+t
            leng=len(stri)
            if(leng!=0):
                if(stri[leng-1]=='_'):
                    stri=stri[0:leng-1]
            
            
            segmented_text.append(stri)
            sentence=[]
            dic={}
        else:
            sentence.append(elem)
  
            
    
   
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    return segmented_text

In [33]:
from nose.tools import assert_equal

# CHECKING THE GENERAL PROPERTIES OF THE OUTPUT
dummy_train_path = "/coursedata/06-subwords/dummy_corpus.txt"

dummy_merges = [('e', 'r'),
                ('er', '_'),
                ('e', 'w'),
                ('n', 'ew'),
                ('l', 'o'),
                ('lo', 'w'),
                ('ma', 'ma')]


# check that the output of the function is a list
assert_equal(type(segment_text(dummy_train_path, dummy_merges)), list)
# check that the output of the function is a list of strings
assert_equal(type(segment_text(dummy_train_path, dummy_merges)[0]), str)


# CHECKING THAT THE FUNCTION IS WORKING AS IT SHOULD
# check that the dummy train corpus is segmented the way it should:

assert_equal(segment_text(dummy_train_path, dummy_merges), ['low',
                                                             'low',
                                                             'low',
                                                             'low',
                                                             'low',
                                                             'low e s t',
                                                             'low e s t',
                                                             'new er',
                                                             'new er',
                                                             'new er',
                                                             'new er',
                                                             'new er',
                                                             'new er',
                                                             'W I D er',
                                                             'W I D er',
                                                             'W I D er',
                                                             'new',
                                                             'new'])


# check that the dummy test corpus is segmented the way it should:
dummy_test_path = "/coursedata/06-subwords/dummy_test_corpus.txt"
assert_equal(segment_text(dummy_test_path, dummy_merges), ['low er', 'c o o l er'])



## TASK 2 <a class="anchor" id="task_2"></a>
## ANALYZE SEGMENTATIONS
## 2.1 <a class="anchor" id="subtask_2_1"></a>
### Count word OOV (1 Point)
Now that we've done with the algorithm, let's see if it will actually help us with the OOV problem.
Let's use the same corpus as we did in the previous POS-tagging assignment. We've randomly shuffled the sentences and split the corpus roughly in half. One half will be our training example, and another will be our test example.

Analyze:
1. The number of word **types** in the vocabulary of the training corpus.
2. The number of word **types** in the vocabulary of the test corpus.
3. The percentage of word **types** in the test corpus vocabulary that are absent in the training corpus. (what part of test corpus vocabulary is OOV?)

Run the cell below, to collect the tokenized corpora (we're splitting the words in the corpora by whitespaces), type in the answer in the next cell.

Here, you can add a new cell and make the computations you need there.

In [34]:
with open("/coursedata/06-subwords/gum_train.txt", 'r') as f:
    train = f.read()
    train = train.split()

with open("/coursedata/06-subwords/gum_test.txt", 'r') as f:
    test = f.read()
    test = test.split()

In [35]:
trainlist=[]
for word in train:
    if(word not in trainlist):
        trainlist.append(word)
print(len(trainlist))


testlist=[]
for word in test:
    if(word not in testlist):
        testlist.append(word)
print(len(testlist))



8677
8770


In [ ]:
# type in the answer as integer number
# For example:
# len_of_train_vocab = 234
#print(counttrain)

len_of_train_vocab = len(trainlist)

# type in the answer as integer number
# For example:
# len_of_test_vocab = 234
len_of_test_vocab = len(testlist)

# type in the answer as a float number between 0 and 100
# For example:
# oov_percentage = 90.9
flist=[]
for w in testlist:
    if(w not in trainlist):
        flist.append(w)

oov_percentage=len(flist)/len(testlist)*100
print(oov_percentage)    

#Remember to remove the raise NotImplementedError line:
# YOUR CODE HERE
#raise NotImplementedError()

In [37]:
from numpy.testing import assert_almost_equal
from nose.tools import assert_equal

# QUESTION 1 tests

# checks if your answer is an integer
assert_equal(type(len_of_train_vocab), int)
# checks if your answer is remorely correct integer
assert(7000 < len_of_train_vocab < 10000)


In [38]:
# QUESTION 2 tests

# checks if your answer is an integer
assert_equal(type(len_of_test_vocab), int)
# checks if your answer is remorely correct integer
assert(7000 < len_of_test_vocab < 10000)


In [39]:
# QUESTION 2 tests

# checks if your answer is a float
assert_equal(type(oov_percentage), float)
# checks if your answer is remorely correct integer
assert(1 < oov_percentage < 70)


## 2.2 <a class="anchor" id="subtask_2_2"></a>
### Count subword OOV (1 Point)

Now, let's compare the OOV numbers we've got in the case where the text is tokenized by words and the case when it's tokenized by subwords. 

Learn 5000 BPE segmentation from the training data, then segment both corpora and compare the vocabulary numbers again. Note that it will take a couple of minutes to run BPE.

Analyze:
1. The number of subword **types** in the vocabulary of the training corpus.
2. The number of subword **types** in the vocabulary of the test corpus.
3. The percentage of subword **types** in the test corpus vocabulary that are absent in the training corpus. (what part of test corpus vocabulary is OOV?)


Again, you can add a new cell and make the computations you need there.

In [ ]:
merges = learn_BPE_merges("/coursedata/06-subwords/gum_train.txt", 5000)
segmented_train = segment_text("/coursedata/06-subwords/gum_train.txt", merges)
segmented_test = segment_text("/coursedata/06-subwords/gum_test.txt", merges)
# represent texts as a list of subwords
segmented_train_as_subwords = " ".join(segmented_train).split(" ")
segmented_test_as_subwords = " ".join(segmented_test).split(" ")


# double checking that your BPE algorithm is working correctly
assert_equal(merges[0], ('e', '_'))
assert_equal(merges[1000], ('t', 'ro'))
assert_equal(merges[-1], ('des', 'cendants_'))

assert_equal(segmented_train[580], 'V ill age')
assert_equal(segmented_train[5400], 'laun ching')

assert_equal(segmented_test[501], 'C a the dr al')
assert_equal(segmented_test[517], 'ho t els')

assert(len(segmented_train_as_subwords) == 64166)
assert(len(segmented_test_as_subwords) == 72712)

In [ ]:
print(len(segmented_test_as_subwords))


In [ ]:
print(segmented_test)


In [ ]:
print(segmented_test_as_subwords)

In [ ]:
strain=[]
stest=[]
for w in segmented_train_as_subwords:
    if(w not in strain):
        strain.append(w)

for w in segmented_test_as_subwords:
    if(w not in stest):
        stest.append(w)

In [ ]:
# type in the answer as an integer number
# For example:
# len_of_train_sub_vocab = 234
len_of_train_sub_vocab = len(strain)
print(len_of_train_sub_vocab)
# type in the answer as an integer number
# For example:
# len_of_test_sub_vocab = 234
len_of_test_sub_vocab = len(stest)
print(len_of_test_sub_vocab)
# type in the answer as a float number between 0 and 100
# For example:
# oov_sub_percentage = 50.9
slist=[]
for w in stest:
    if (w not in strain):
        slist.append(w)
oov_sub_percentage  = len(slist)/len(stest)*100
print(oov_sub_percentage)
#Remember to remove the raise NotImplementedError line:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# QUESTION 1 tests

# checks if your answer is an integer
assert_equal(type(len_of_train_sub_vocab), int)
# checks if your answer is remorely correct integer
assert(500 < len_of_train_sub_vocab < 5000)


In [ ]:
# QUESTION 2 tests

# checks if your answer is an integer
assert_equal(type(len_of_test_sub_vocab), int)
# checks if your answer is remorely correct integer
assert(500 < len_of_test_sub_vocab < 5000)


In [ ]:
# QUESTION 3 tests

# checks if your answer is a float
assert_equal(type(oov_sub_percentage), float)
# checks if your answer is remorely correct integer
assert(1 < oov_sub_percentage < 50)


## 2.3 <a class="anchor" id="subtask_2_3"></a>
### Does the segmentation make sense? (1 Point)
Now let's look a bit closer at the subwords that we've learned. Take a second to think if the results are what you would expect them to be.

1. What are the top 10 most frequent subwords in the segmented test corpus? (in decending frequency order)
2. What are the top 5 longest subwords in the segmented test corpus? (sort alphabetically)
3. What are the top 5 most frequent lengths of subwords in the test corpus?

In [ ]:
# type in the answer as a list of strings
# For example:
# top_10_by_freq = ['a','b'...]
dic=collections.Counter(segmented_test_as_subwords)

i=0
lista=[]

for k in dic:
    if(i==10):
        break;
    lista.append(k)
    i=i+1
top_10_by_freq = ['', 's', 't', 'k', 'a', 'ed', 'o', 'e', 'es', 'p']
print(len(top_10_by_freq))




   
lista=[]
for k in dic:
    lista.append(k)
lst2=sorted(lista, key=len, reverse=True)
top_5_by_len = lst2[0:5]
print(top_5_by_len)
# type in the answer as a list of integers
# For example:
# top_5_freqs_of_lens= [1,2,3...]

lista=[]
for k in dic:
    for i in range(dic[k]):
        lista.append(len(k))

coll=collections.Counter(lista)
print(coll)
top_5_freqs_of_lens = ['0','2','1','3','4']
print(top_5_freqs_of_lens)
#Remember to remove the raise NotImplementedError line:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# QUESTION 1 tests

# checks if your answer is a list of strings
assert_equal(type(top_10_by_freq), list)
assert_equal(type(top_10_by_freq[0]), str)
assert_equal(len(top_10_by_freq), 10)


In [ ]:
# QUESTION 2 tests

# checks if your answer is a list of strings
assert_equal(type(top_5_by_len), list)
assert_equal(type(top_5_by_len[0]), str)
assert_equal(len(top_5_by_len), 5)


In [ ]:
# QUESTION 3 tests

# checks if your answer is a list of integers
assert_equal(type(top_5_freqs_of_lens), list)
assert_equal(type(top_5_freqs_of_lens[0]), int)
assert_equal(len(top_5_freqs_of_lens), 5)



## 2.4 <a class="anchor" id="subtask_2_4"></a>
### Your thoughts (3 Points)
Briefly answer the following questions:

1. Describe what happens when you change the k parameter? How to find a good number for k?

2. What are the possible NLP applications that can benefit from the tokenization into subwords? Why?

3. How the OOV number for our data can be lowered further without changing anything in the segmentation procedure (k stays the same)? Hint: think about the things we used to do in the previous assignments but are not doing in this one.

1. If we vary the k-parameter the number of merges found changes. In particular for our application above subwords with lower length are retrieved in case the k decrease otherwise subwords with higher length are found. A good number of k can be found imposing as a limit that a merge should appear at least 2 times for example. So a tuple of two elements is considered a merge if for example appears 2 times in the text. (2 is an arbitrary number). So when the only merges found appears in x occurences we stop the search.
2. For example text classification. In text classification the meaning of a word can be exploited by the meaning of its subwords and consequently we can classify a text based on this. Text translation: suppose that we don't have the translation for a compound word: we can translate it with the translation of of its subwords.
3. Lemmatization

## Checklist before submission <a class="anchor" id="checklist"></a>
### 1
To make sure that you didn't forget to import some package or to name some variable, press **Kernel -> Restart** and then **Cell -> Run All**. This way your code will be run exactly in the same order as during the autograding.
### 2
Click the **Validate** button in the upper menu to check that you haven't missed anything. You might need to run the validation in the terminal because BPE algorithm takes time.
### 3
To submit the notebook, click on the **jupyterhub** logo in the upper left part of the window, choose the **Assignments** folder, and press **submit**. You can submit multiple times, only the last one counts.
### 4
Please fill in the feedback form in the [Assignment](https://mycourses.aalto.fi/mod/questionnaire/view.php?id=984937) section of Mycoures.